In [286]:
import pandas as pd
from miditime.miditime import MIDITime
import numpy as np
import csv
import json

In [287]:
#Read in CSV, normalize coal production data (to millions)
df = pd.read_csv('coal_prod_1984_2016_weeks_summed.csv', encoding='utf8', names=["Year", "Week", "CoalProd"])
df["CoalProdMillions"]=df[["CoalProd"]]/1000000

In [288]:
# data processing if you want to listen to the data monthly
def process_data_months(df, csvfile):
    # Remove weeks 1 and weeks 53
    df = df[df.Week != 53]
    df = df[df.Week != 1]
    df = df[df.Week != 52]
    df = df[df.Week != 27]
    #reset index and select just index and coal production (millions) columns
    df.reset_index(inplace=True)

    # Bin into groups of four weeks
    df['Month'] = np.floor(df.index/4)

    df2 = df.groupby('Month').aggregate(sum)

    #Export dataframe as CSV
    df2.to_csv(csvfile, header=False)

In [289]:
# data processing if you want to listen to the data weekly
def process_data_weeks(df, csvfile):
    # Remove weeks 1 and weeks 53
    df = df[df.Week != 53]
    df = df[df.Week != 1]
    df = df[df.Week != 52]
    df = df[df.Week != 27]
    #reset index and select just index and coal production (millions) columns
    df.reset_index(inplace=True)

    #Export dataframe as CSV
    df.to_csv(csvfile, header=False)

In [290]:
process_data_weeks(df,'coal_sonify_weeks.csv')
process_data_months(df,'coal_sonify_months.csv')

In [291]:
def csv_to_MIDITime_data(filename):
    mydata = []
    with open(filename, 'rb') as f:
        reader=csv.reader(f)
        for row in reader:
            mydict = {'days_since_epoch': int(float(row[0])) , 'magnitude': float(row[5])}
            mydata.append(mydict)
    return mydata

In [292]:
mydata = csv_to_MIDITime_data('coal_sonify_weeks.csv')

mymidi = MIDITime(80, 'myfile.mid', 45, 2, 5)

my_data_timed = [{'beat': mymidi.beat(d['days_since_epoch']), 'magnitude': d['magnitude']} for d in mydata]

start_time = my_data_timed[0]['beat']

In [293]:
def mag_to_pitch_tuned(magnitude):
    # Where does this data point sit in the domain of your data? (I.E. the min magnitude is 3, the max in 5.6). In this case the optional 'True' means the scale is reversed, so the highest value will return the lowest percentage.
    scale_pct = mymidi.linear_scale_pct(10, 25, magnitude)

    # Another option: Linear scale, reverse order
    # scale_pct = mymidi.linear_scale_pct(3, 5.7, magnitude, True)

    # Another option: Logarithmic scale, reverse order
    # scale_pct = mymidi.log_scale_pct(3, 5.7, magnitude, True)

    # Pick a range of notes. This allows you to play in a key.
    c_major = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

    #Find the note that matches your data point
    note = mymidi.scale_to_note(scale_pct, c_major)

    #Translate that note to a MIDI pitch
    midi_pitch = mymidi.note_to_midi_pitch(note)

    return midi_pitch

In [294]:
def mag_to_attack(magnitude):
    # Where does this data point sit in the domain of your data? (I.E. the min magnitude is 3, the max in 5.6). In this case the optional 'True' means the scale is reversed, so the highest value will return the lowest percentage.
    scale_pct = mymidi.linear_scale_pct(10, 25, magnitude)
    
    #max_attack = 10

    adj_attack = (1-scale_pct)*max_attack + 70
    #adj_attack = 100

    return adj_attack

In [295]:
note_list = []

for d in my_data_timed:
    note_list.append([
        d['beat'] - start_time,
        mag_to_pitch_tuned(d['magnitude']),
        100,
        #mag_to_attack(d['magnitude']),  # attack
        0.5  # duration, in beats
    ])

In [296]:
# Add a track with those notes
mymidi.add_track(note_list)

# Output the .mid file
mymidi.save_midi()

29 0.0 0.5 100
